In [1]:
# imports
import pandas as pd
import numpy as np
import re

In [2]:
# inputs
test_data = open('input-test.txt').read()
data = open('input.txt').read()

# Part 1

In [3]:
# test
content = test_data

def parse_input(content):
    template, insertion_rules = content.split('\n\n')
    insertion_rules = dict(re.findall('([A-Z]+) -> ([A-Z]+)', insertion_rules))
    return template, insertion_rules


def find_matches(element, substitute, template):
    matches = []
    for idx in range(len(template)):
        if ''.join(template[idx:idx+2]) == element:
            matches.append((substitute, idx))
    
    return matches


def polymerize(content, n_steps = 1):
    template, insertion_rules = parse_input(content)

    while n_steps > 0:
        matches = []
        for k, v in insertion_rules.items():
            matches.extend(find_matches(k, v, template))

        matches = sorted(matches, key=lambda x: -x[1])

        template = np.array(list(template))
        for sub, idx in matches:
            template = np.insert(template, idx+1, sub)

        template = ''.join(template)
        n_steps -= 1

    return template


def extract_result(polymer):
    _, counts = np.unique(list(polymer), return_counts=True)
    return max(counts) - min(counts)


assert polymerize(content, 1) == 'NCNBCHB'
assert polymerize(content, 2) == 'NBCCNBBBCBHCB'
assert polymerize(content, 3) == 'NBBBCNCCNBBNBNBBCHBHHBCHB'
assert polymerize(content, 4) == 'NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB'
assert extract_result(polymerize(content, 10)) == 1588

In [4]:
# real
content = data
extract_result(polymerize(content, 10))

2703

# Part 2

In [5]:
# test
content = test_data

def get_pairs(template):
    return [''.join(template[idx:idx+2]) for idx in range(len(template)-1)]


def get_pair_counts(template):
    pairs = get_pairs(template)
    return {pair: len(find_matches(pair, None, template)) for pair in pairs}


def extract_result_fast(content, n_steps = 1):
    template, insertion_rules = parse_input(content)

    pairs = get_pairs(template)
    pair_counts = {pair: len(find_matches(pair, None, template)) for pair in pairs}
    el_counts = {k: int(v) for (k, v) in np.transpose(np.unique(list(template), return_counts=True))}

    while n_steps > 0:

        occurrences = {el: pair_counts[el] for el, _ in insertion_rules.items() if el in pair_counts}
        for el, count in occurrences.items(): 
            del pair_counts[el]

        for el, count in occurrences.items(): 
            sub = insertion_rules[el]
            el_counts[sub] = 0 if sub not in el_counts else el_counts[sub]
            el_counts[sub] += count
            new_elements = [f'{el[0]}{sub}', f'{sub}{el[1]}']
            for new_el in new_elements:
                pair_counts[new_el] = 0 if new_el not in pair_counts else pair_counts[new_el]
                pair_counts[new_el] += count
                
        n_steps -= 1

    return max(el_counts.values()) - min(el_counts.values())

assert extract_result_fast(content, 10) == 1588
assert extract_result_fast(content, 40) == 2188189693529

In [6]:
# test
content = data
extract_result_fast(content, 40)

2984946368465